In [23]:
import os

item_map = {
    'Realism': [],
    'Rococo': []
}

item_to_idx = {
    'Realism': 0,
    'Rococo': 0
}

data_dir = '/projects/data/wikiart/'
counter = 0
for category in item_map.keys():
    fnames = os.listdir(data_dir + category)[0:1000]
    
    for fn in fnames:
        item_map[category].append({
            'idx': counter,
            'label': item_to_idx[category],
            'filename': category + '/' + fn,
        })
        counter += 1
    
for k in item_map:
    print(k, len(item_map[k]))

Realism 1000
Rococo 1000


In [24]:
import random
for category in item_map.keys():
    random.shuffle(item_map[category])

In [25]:
def write_lst(image_arr, base_dir, file_path):
    with open(file_path, 'w') as f:
        count = 0
        for img in image_arr:
            label = img['label']
            img_path = os.path.join(base_dir, img['filename'])
            new_line = '\t'.join([str(count), str(label), str(img_path)])
            new_line += '\n'
            f.write(new_line)
            count += 1

In [26]:
import sys
min_data_len = sys.maxsize

for category in item_map.keys():
    min(min_data_len, len(item_map[category]))

sample = (0, 8)
train = (0, int(min_data_len * 0.7))
validation = (int(min_data_len * 0.7), int(min_data_len * 0.85))
test = (int(min_data_len * 0.85), int(min_data_len * 1))

def split_dataset(from_idx, to_idx):
    result = []
    for category in item_map.keys():
        result = result + item_map[category][from_idx: to_idx]
    
    return result

In [27]:
# sample set is for developing model and debugging
# because debugging with large dataset takes a long time
sample_set = split_dataset(sample[0], sample[1])
write_lst(sample_set, '/projects/data/wikiart/', '/projects/data/wikiart/sample.lst')

train_set = split_dataset(train[0], train[1])
write_lst(train_set, '/projects/data/wikiart/', '/projects/data/wikiart/train.lst')

validation_set = split_dataset(validation[0], validation[1])
write_lst(validation_set, '/projects/data/wikiart/', '/projects/data/wikiart/validation.lst')

test_set = split_dataset(test[0], test[1])
write_lst(validation_set, '/projects/data/wikiart/', '/projects/data/wikiart/test.lst')


In [28]:
from mxnet.gluon.model_zoo.vision import mobilenet1_0
pretrained_net = mobilenet1_0(pretrained=True)
print(pretrained_net)


MobileNet(
  (features): HybridSequential(
    (0): Conv2D(3 -> 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=32)
    (2): Activation(relu)
    (3): Conv2D(1 -> 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (4): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=32)
    (5): Activation(relu)
    (6): Conv2D(32 -> 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
    (8): Activation(relu)
    (9): Conv2D(1 -> 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
    (10): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
    (11): Activation(relu)
    (12): Conv2D(64 -> 128, kernel_size=(1, 1), stride=(

In [29]:
net = mobilenet1_0(classes=2)

In [30]:
from mxnet import init
net.features = pretrained_net.features
net.output.initialize(init.Xavier())


In [31]:
from mxnet.image import color_normalize
from mxnet import image
train_augs = [
    image.ResizeAug(224),
    image.HorizontalFlipAug(0.5),  # flip the image horizontally
    image.BrightnessJitterAug(.3), # randomly change the brightness
    image.HueJitterAug(.1)         # randomly change hue
]
test_augs = [
    image.ResizeAug(224)
]
def transform(data, label, augs):
    data = data.astype('float32')
    for aug in augs:
        data = aug(data)
    data = nd.transpose(data, (2,0,1))
    return data, nd.array([label]).asscalar().astype('float32')


In [33]:
from mxnet.gluon.data.vision import ImageRecordDataset
train_rec = '/projects/data/wikiart/train.rec' # train
validation_rec = '/projects/data/wikiart/validation.rec'
trainIterator = ImageRecordDataset(
    filename=train_rec, 
    transform=lambda X, y: transform(X, y, train_augs)
)

validationIterator = ImageRecordDataset(
    filename=validation_rec,
    transform=lambda X, y: transform(X, y, test_augs)
)

In [34]:
def train(net, ctx, 
          batch_size=64, epochs=10, learning_rate=0.01, wd=0.001):
    train_data = gluon.data.DataLoader(
        trainIterator, batch_size, shuffle=True)
    validation_data = gluon.data.DataLoader(
        validationIterator, batch_size)
    net.collect_params().reset_ctx(ctx)
    net.hybridize()
    
    loss = gluon.loss.SoftmaxCrossEntropyLoss()
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {
        'learning_rate': learning_rate, 'wd': wd})
    
    train_util(net, train_data, validation_data, 
               loss, trainer, ctx, epochs, batch_size)

In [35]:
import time
from mxnet.image import color_normalize
from mxnet import autograd
import mxnet as mx
from mxnet import nd
from mxnet import gluon
import json

def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx) # a big array
        label = label.as_in_context(ctx) # "accuracy"
        data = color_normalize(data/255,
                               mean=mx.nd.array([0.485, 0.456, 0.406]).reshape((1,3,1,1)),
                               std=mx.nd.array([0.229, 0.224, 0.225]).reshape((1,3,1,1)))
        output = net(data) # an array
        prediction = nd.argmax(output, axis=1) # an NDArray
        acc.update(preds=prediction, labels=label)
    
    print(json.dumps(acc.get()))
    
    return acc.get()[1]

def metric_str(names, accs):
    return ', '.join(['%s=%f'%(name, acc) for name, acc in zip(names, accs)])

def train_util(net, train_iter, validation_iter, loss_fn, trainer, ctx, epochs, batch_size):
    metric = mx.metric.create(['acc'])
    for epoch in range(epochs):
        for i, (data, label) in enumerate(train_iter):
            st = time.time()
            # ensure context            
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            # normalize images
            data = color_normalize(data/255,
                                   mean=mx.nd.array([0.485, 0.456, 0.406]).reshape((1,3,1,1)),
                                   std=mx.nd.array([0.229, 0.224, 0.225]).reshape((1,3,1,1)))
            
            with autograd.record():
                output = net(data)
                loss = loss_fn(output, label)

            loss.backward()
            trainer.step(data.shape[0])
            
            #  Keep a moving average of the losses
            metric.update([label], [output])
            names, accs = metric.get()
            print('[Epoch %d Batch %d] speed: %f samples/s, training: %s'%(epoch, i, batch_size/(time.time()-st), metric_str(names, accs)))
            if i%100 == 0:
                net.collect_params().save('./checkpoints/%d-%d.params'%(epoch, i))

        train_acc = evaluate_accuracy(train_iter, net)
        validation_acc = evaluate_accuracy(validation_iter, net)
        print("Epoch %s | training_acc %s | val_acc %s " % (epoch, train_acc, validation_acc))

In [ ]:
import mxnet as mx
import datetime

print(datetime.datetime.now())
ctx = mx.cpu()
train(net, ctx, batch_size=64, epochs=5, learning_rate=0.003)

print(datetime.datetime.now())

2018-11-07 07:20:35.888868
[Epoch 0 Batch 0] speed: 15.328608 samples/s, training: accuracy=0.109375
[Epoch 0 Batch 1] speed: 15.436512 samples/s, training: accuracy=0.445312
[Epoch 0 Batch 2] speed: 15.397572 samples/s, training: accuracy=0.614583
[Epoch 0 Batch 3] speed: 15.395245 samples/s, training: accuracy=0.710938
[Epoch 0 Batch 4] speed: 15.403290 samples/s, training: accuracy=0.768750
[Epoch 0 Batch 5] speed: 15.353210 samples/s, training: accuracy=0.807292
[Epoch 0 Batch 6] speed: 15.380855 samples/s, training: accuracy=0.834821
[Epoch 0 Batch 7] speed: 15.345632 samples/s, training: accuracy=0.855469
[Epoch 0 Batch 8] speed: 15.303424 samples/s, training: accuracy=0.871528
[Epoch 0 Batch 9] speed: 15.277411 samples/s, training: accuracy=0.884375
[Epoch 0 Batch 10] speed: 15.243418 samples/s, training: accuracy=0.894886
[Epoch 0 Batch 11] speed: 14.181223 samples/s, training: accuracy=0.903646
[Epoch 0 Batch 12] speed: 15.337754 samples/s, training: accuracy=0.911058
[Epoch 0

In [ ]:
import mxnet as mx
import datetime
print(datetime.datetime.now())
ctx2 = mx.gpu()
train(net, ctx2, batch_size=64, epochs=5, learning_rate=0.003)
print(datetime.datetime.now())

2018-11-07 07:12:03.860191


In [ ]:
#todo - another set of timings with mkl

In [ ]:
test_data_loader = gluon.data.DataLoader(testIterator, 64)
test_acc = evaluate_accuracy(test_data_loader, net)
